In [1]:

import soundfile as sf
import csv
import time
import threading
import sounddevice as sd
import numpy as np
from pynput import keyboard
from scipy.io.wavfile import write as wav_write

### The following code is used to record the audio and the keystrokes

*It is not used in the final project, but it is kept here for reference
*The audio recording will go on for 10 seconds. The keystroke recording will be done simultaneously but needs to be stopped manually by pressing ESC (escape)

Head's up : Dont forget to grant accessibility access the editor with which you're running the code below, in particular input monitoring for keystroke recording.


In [2]:
import numpy as np
import csv
import time
import sounddevice as sd
import threading
import scipy.io.wavfile as wav
from pynput import keyboard

# Parameters for sound recording
sample_rate = 44100  # Hz
duration = 60  # seconds
audio_data = np.zeros((duration * sample_rate, 1), dtype=np.int16)

# File paths
audio_file = 'keyboard_sound.wav'
log_file = 'key_log.csv'

# Initialize the keystroke log file
with open(log_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Key', 'Action', 'Timestamp'])

# Global variable to track the start time of audio recording
start_time_audio = None
stop_recording = False  # Flag to stop both audio and keyboard listeners

# Recording function for audio
def record_audio():
    global audio_data, start_time_audio
    print("Recording audio...")

    # Set the start time of the recording
    start_time_audio = time.time()  # Capture the timestamp when recording starts

    # Record the audio and store it in audio_data
    try : 
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype='int16')
        sd.wait()
        wav.write(audio_file, sample_rate, audio_data)
        print(f"Audio saved to {audio_file}")
    except:
        print("Your computer does not support stereo recording. Defaulting to mono.")
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
        sd.wait()
        wav.write(audio_file, sample_rate, audio_data)
        print(f"Audio saved to {audio_file}")

    print("Audio recording finished")

    # Save the recorded audio to a WAV file

# Keystroke listener function
def on_press(key):
    if start_time_audio is None:
        return  # Don't log if the audio hasn't started yet

    timestamp = time.time() - start_time_audio  # Calculate relative timestamp
    try:
        key_str = key.char  # Normal keys
    except AttributeError:
        key_str = str(key)  # Special keys like shift, ctrl, etc.

    # Log the key press with relative timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Pressed', round(timestamp, 6)])  # Round for cleaner timestamp

    print(f"Key {key_str} Pressed at {timestamp:.6f} seconds")

def on_release(key):
    if start_time_audio is None:
        return  # Don't log if the audio hasn't started yet

    timestamp = time.time() - start_time_audio  # Calculate relative timestamp
    try:
        key_str = key.char
    except AttributeError:
        key_str = str(key)

    # Log the key release with relative timestamp
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([key_str, 'Released', round(timestamp, 6)])  # Round for cleaner timestamp

    print(f"Key {key_str} Released at {timestamp:.6f} seconds")

    # Stop listener if 'Esc' key is pressed
    if key == keyboard.Key.esc:
        global stop_recording
        stop_recording = True  # Set flag to stop both recordings
        return False

# Start recording audio in a separate thread
audio_thread = threading.Thread(target=record_audio)
audio_thread.start()

# Start the keyboard listener in the main thread to avoid blocking
def start_keyboard_listener():
    with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()

# Run the keyboard listener in a separate thread
keyboard_thread = threading.Thread(target=start_keyboard_listener)
keyboard_thread.start()

# Wait for both threads to finish, while checking for stop condition
while not stop_recording:
    time.sleep(0.1)  # Prevent high CPU usage by sleeping briefly

# Once 'Esc' is pressed, both threads will finish
audio_thread.join()
keyboard_thread.join()
print("Recording process finished.")

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


Recording audio...
Your computer does not support stereo recording. Defaulting to mono.
Key q Pressed at 4.593025 seconds
Key q Released at 4.747319 seconds
Key w Pressed at 5.084450 seconds
Key w Released at 5.260164 seconds
Key e Pressed at 5.644337 seconds
Key e Released at 5.776860 seconds
Key r Pressed at 6.147102 seconds
Key r Released at 6.310367 seconds
Key t Pressed at 6.651508 seconds
Key t Released at 6.814377 seconds
Key y Pressed at 7.170338 seconds
Key y Released at 7.340303 seconds
Key u Pressed at 7.678346 seconds
Key u Released at 7.858778 seconds
Key i Pressed at 8.214250 seconds
Key i Released at 8.368478 seconds
Key o Pressed at 8.764603 seconds
Key o Released at 8.942689 seconds
Key p Pressed at 9.297351 seconds
Key p Released at 9.466547 seconds
Key a Pressed at 10.251167 seconds
Key a Released at 10.401312 seconds
Key s Pressed at 10.793203 seconds
Key s Released at 10.947356 seconds
Key d Pressed at 11.334717 seconds
Key d Released at 11.469191 seconds
Key f Pre

In [7]:
print(sd.query_devices())

> 0 MacBook Pro Microphone, Core Audio (1 in, 0 out)
< 1 MacBook Pro Speakers, Core Audio (0 in, 2 out)
  2 Microsoft Teams Audio, Core Audio (1 in, 1 out)


In [3]:
# Load the audio file
audio_path = "keyboard_sound.wav"
audio_data, sample_rate = sf.read(audio_path)

# Read keystroke timestamps from CSV
keystroke_times = []
with open("key_log.csv", "r") as audio_data_file:
    reader = csv.reader(audio_data_file)
    stack = {}

    for row in reader:
        if row[0] == "Key":
            continue
        key = row[0]
        action = row[1]
        timestamp = float(row[2])

        if action == "Pressed":
            if key in stack:
                raise Exception(f"Key {key} was pressed again before being released.")
            stack[key] = [key, timestamp]

        elif action == "Released":
            if key not in stack:
                raise Exception(f"Key {key} was released without being pressed.")
            stack[key].append(timestamp)
            keystroke_times.append(stack[key])
            del stack[key]

print(keystroke_times)


[['q', 4.593025, 4.747319], ['w', 5.08445, 5.260164], ['e', 5.644337, 5.77686], ['r', 6.147102, 6.310367], ['t', 6.651508, 6.814377], ['y', 7.170338, 7.340303], ['u', 7.678346, 7.858778], ['i', 8.21425, 8.368478], ['o', 8.764603, 8.942689], ['p', 9.297351, 9.466547], ['a', 10.251167, 10.401312], ['s', 10.793203, 10.947356], ['d', 11.334717, 11.469191], ['f', 11.834781, 12.013504], ['g', 12.32099, 12.491275], ['h', 12.834766, 12.992994], ['j', 13.311012, 13.451964], ['k', 13.779466, 13.924242], ['l', 14.280903, 14.427964], ['z', 14.958398, 15.089112], ['x', 15.408239, 15.59492], ['c', 15.869536, 16.03884], ['v', 16.302168, 16.459244], ['b', 16.735, 16.911174], ['n', 17.162669, 17.305524], ['m', 17.589113, 17.737113], ['d', 18.623912, 18.745636], ['f', 19.037925, 19.173545], ['t', 19.478426, 19.615561], ['g', 19.89209, 20.064431], ['v', 20.338471, 20.483414], ['f', 20.765368, 20.881692], ['s', 21.263501, 21.393394], ['d', 21.684501, 21.811857], ['r', 22.102582, 22.236903], ['t', 22.52181

## The code that generates individual spectrograms for each keystroke

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import scipy.signal as signal
import csv
import os

# Parameters
AUDIO_FILE = "keyboard_sound.wav"  # Replace with your actual audio file
KEYSTROKE_CSV = "key_log.csv"
OUTPUT_DIR = "keystroke_spectrograms/keystroke_spectrograms"
NUMPY_OUTPUT_DIR = "keystroke_spectrograms/numpy_arrays"  # New directory for NumPy arrays

## Hyperparameter to fine-tune
BUFFER_TIME = 0.2  # Extra time (seconds) before & after each keystroke

# Ensure output directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(NUMPY_OUTPUT_DIR, exist_ok=True)

# Load audio data
sample_rate, audio_data = wav.read(AUDIO_FILE)

# Read keystroke data from CSV
keystroke_times = []
with open(KEYSTROKE_CSV, "r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    stack = {}

    for row in reader:
        key, action, timestamp = row[0], row[1], float(row[2])

        if action == "Pressed":
            stack[key] = timestamp  # Store press time
        elif action == "Released" and key in stack:
            keystroke_times.append((key, stack.pop(key), timestamp))  # Store key, press, and release

# Function to create and save the spectrogram and numpy arrays
def create_spectrogram_and_numpy(audio_segment, key, idx):
    # Generate the spectrogram using scipy
    f, t, Sxx = signal.spectrogram(audio_segment, sample_rate)

    # Normalize spectrogram for neural network (optional)
    Sxx_log = 10 * np.log10(Sxx + 1e-10)  # Adding small constant to avoid log(0)

    # Plot the spectrogram
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(t, f, Sxx_log, shading='auto', cmap='inferno')
    plt.colorbar(label='Power (dB)')
    plt.title(f"Keystroke '{key}' Spectrogram")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")

    # Save the spectrogram to file as PNG
    spectrogram_path = os.path.join(OUTPUT_DIR, f"keystroke_{idx + 1}_{key}.png")
    plt.savefig(spectrogram_path)
    plt.close()
    print(f"Saved spectrogram for '{key}' at {spectrogram_path}")

    # Save the spectrogram as a NumPy array
    numpy_array_path = os.path.join(NUMPY_OUTPUT_DIR, f"keystroke_{idx + 1}_{key}.npy")
    np.save(numpy_array_path, Sxx_log)  # Save the log-scaled spectrogram as a numpy array
    print(f"Saved NumPy array for '{key}' at {numpy_array_path}")

# Process keystrokes
for idx, (key, press_time, release_time) in enumerate(keystroke_times):
    # Define start and end times (add buffer)
    start_time = max(0, press_time - BUFFER_TIME)
    end_time = min(len(audio_data) / sample_rate, release_time + BUFFER_TIME)

    # Convert times to sample indices
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)

    # Extract audio segment
    keystroke_audio = audio_data[start_sample:end_sample]

    if len(keystroke_audio) == 0:
        print(f"Warning: Empty audio segment for keystroke {idx + 1}")
        continue

    # Create and save the spectrogram and numpy array for this key
    create_spectrogram_and_numpy(keystroke_audio, key, idx)

print("Processing complete. Spectrograms and NumPy arrays saved.")


Saved spectrogram for 'q' at keystroke_spectrograms/keystroke_spectrograms/keystroke_1_q.png
Saved NumPy array for 'q' at keystroke_spectrograms/numpy_arrays/keystroke_1_q.npy
Saved spectrogram for 'w' at keystroke_spectrograms/keystroke_spectrograms/keystroke_2_w.png
Saved NumPy array for 'w' at keystroke_spectrograms/numpy_arrays/keystroke_2_w.npy
Saved spectrogram for 'e' at keystroke_spectrograms/keystroke_spectrograms/keystroke_3_e.png
Saved NumPy array for 'e' at keystroke_spectrograms/numpy_arrays/keystroke_3_e.npy
Saved spectrogram for 'r' at keystroke_spectrograms/keystroke_spectrograms/keystroke_4_r.png
Saved NumPy array for 'r' at keystroke_spectrograms/numpy_arrays/keystroke_4_r.npy
Saved spectrogram for 't' at keystroke_spectrograms/keystroke_spectrograms/keystroke_5_t.png
Saved NumPy array for 't' at keystroke_spectrograms/numpy_arrays/keystroke_5_t.npy
Saved spectrogram for 'y' at keystroke_spectrograms/keystroke_spectrograms/keystroke_6_y.png
Saved NumPy array for 'y' a